# Prática: Árvores de Decisão


### Objetivo
Para minimizar as perdas de um banco fictício, precisamos desenvolver um processo de tomada de decisão sobre para quem o banco deve aprovar empréstimos e para quem não. Os perfis demográfico e socioeconômico do cliente são considerados pelos fictícios gerentes de empréstimos antes da tomada de decisão sobre o pedido de empréstimo.

Com base na base de dados de clientes que pegaram empréstimos no banco fictício, temos classificados os clientes inadimplentes e os clientes e quitaram as suas dívidas.

Nosso objetivo é construir um Modelo de Machine Learning que irá prever se um cliente que aplica para um empréstimo pode ser ou não um cliente inadimplente.

Este tutorial é fornecido pela equipe da BRAINS, para acessar a prática completa clique a seguir: [Prática: Árvores de Decisão - BRAIS - Brasilian AI Networks](https://brains.dev/2023/pratica-arvores-de-decisao/)

## Importanção das Bibliotecas

In [1]:
# Manipulação de dados
import numpy as np
import pandas as pd

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# Divisão dos dados
from sklearn.model_selection import train_test_split

# Algoritmos de Machine Learning
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

# Métricas de performance
from sklearn import metrics
from sklearn.metrics import (f1_score,
                            accuracy_score,
                            recall_score,
                            precision_score,
                            confusion_matrix,
                            plot_confusion_matrix,
                            roc_auc_score)
 
# Ajustes de Hiperparametros
from sklearn.model_selection import GridSearchCV
 
# Optional para Annotations das funções
from typing import Optional
 
# Ignorar alertas
import warnings
warnings.filterwarnings('ignore')

## Carregamento e Manipulação inicial dos Dados

In [3]:
# Carregar os dados em um DataFrame
df = pd.read_csv('credito.csv')
df.head()

,saldo_corrente,duracao_emp_meses,historico_credito,motivo,quantia,saldo_poupanca,tempo_empregado,porcentagem_renda,anos_residencia,idade,outro_credito,residencia,qtd_emprestimos_existentes,emprego,dependentes,telefone,inadimplente
0,< 0 DM,6.0,critico,moveis/eletrodomesticos,1169,desconhecido,> 7 anos,4.0,4.0,67,nenhum,propria,2,qualificado,1,sim,nao
1,1 - 200 DM,48.0,bom,moveis/eletrodomesticos,5951,< 100 DM,1 - 4 anos,2.0,2.0,22,nenhum,propria,1,qualificado,1,nao,sim
2,desconhecido,12.0,critico,educacao,2096,< 100 DM,4 - 7 anos,2.0,3.0,49,nenhum,propria,1,nao-qualificado,2,nao,nao
3,< 0 DM,42.0,bom,moveis/eletrodomesticos,7882,< 100 DM,4 - 7 anos,2.0,4.0,45,nenhum,outros,1,qualificado,2,nao,nao
4,< 0 DM,24.0,ruim,carro,4870,< 100 DM,1 - 4 anos,3.0,4.0,53,nenhum,outros,2,qualificado,2,nao,sim


In [4]:
# Verificar o formato dos dados
df.shape

(1000, 17)

In [5]:
# Resumo estatístico
df.describe()

,duracao_emp_meses,quantia,porcentagem_renda,anos_residencia,idade,qtd_emprestimos_existentes,dependentes
count,996.000000,1000.000000,994.000000,997.000000,1000.000000,1000.000000,1000.000000
mean,20.914659,3271.258000,2.974849,2.844534,35.546000,1.407000,1.155000
std,12.081621,2822.736876,1.120338,1.105346,11.375469,0.577654,0.362086
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000


In [6]:
# Verificar tipos de colunas e quantidade de entradas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   saldo_corrente              1000 non-null   object 
 1   duracao_emp_meses           996 non-null    float64
 2   historico_credito           1000 non-null   object 
 3   motivo                      1000 non-null   object 
 4   quantia                     1000 non-null   int64  
 5   saldo_poupanca              1000 non-null   object 
 6   tempo_empregado             1000 non-null   object 
 7   porcentagem_renda           994 non-null    float64
 8   anos_residencia             997 non-null    float64
 9   idade                       1000 non-null   int64  
 10  outro_credito               1000 non-null   object 
 11  residencia                  1000 non-null   object 
 12  qtd_emprestimos_existentes  1000 non-null   int64  
 13  emprego                     1000 n

In [10]:
# Verificar as colunas com dados nulos
display(df.isnull().sum()[df.isnull().sum() > 0])

duracao_emp_meses    4
porcentagem_renda    6
anos_residencia      3
dtype: int64